In [1]:
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalizationV2

In [2]:
train_path = '../data/blur_single/seg_train/'
test_path = '../data/blur_single/seg_test/'

In [3]:
train_datagen = ImageDataGenerator(
                    rescale = 1/255,
                    shear_range = 0.2,
                    zoom_range = 0.2,
                    horizontal_flip = True,
                    vertical_flip = False)
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (64, 64),
                                                 class_mode = 'categorical')

Found 14034 images belonging to 6 classes.


In [4]:
test_datagen = ImageDataGenerator(rescale = 1/255)
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (64, 64),
                                            class_mode = 'categorical')

Found 3000 images belonging to 6 classes.


In [5]:
cnn = Sequential()
cnn

Metal device set to: Apple M1


2022-08-29 22:43:20.184312: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-29 22:43:20.184463: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
cnn.add(Convolution2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Convolution2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Convolution2D(filters=128, kernel_size=3, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Convolution2D(filters=256, kernel_size=3, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Dropout(0.4)),

cnn.add(Flatten())

cnn.add(Dense(units=256, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(Dropout(0.25))

cnn.add(Dense(units=128, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(Dropout(0.25))

cnn.add(Dense(units=64, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(Dropout(0.25))

cnn.add(Dense(units=24, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(Dropout(0.25))

cnn.add(Dense(units=6, activation='softmax'))

In [7]:
cnn.compile(optimizer = 'sgd', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [8]:
%%time
cnn.fit(training_set, steps_per_epoch= len(training_set), validation_data = test_set, validation_steps=len(test_set), epochs = 10)

Epoch 1/10


2022-08-29 22:43:20.702919: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-29 22:43:21.693047: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


439/439 [==============================] - ETA: 0s - loss: 1.5975 - accuracy: 0.3823

2022-08-29 22:43:50.897975: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


439/439 [==============================] - 33s 70ms/step - loss: 1.5975 - accuracy: 0.3823 - val_loss: 1.4621 - val_accuracy: 0.3997
Epoch 2/10
439/439 [==============================] - 30s 68ms/step - loss: 1.2029 - accuracy: 0.5098 - val_loss: 1.0643 - val_accuracy: 0.5750
Epoch 3/10
439/439 [==============================] - 28s 63ms/step - loss: 1.1102 - accuracy: 0.5600 - val_loss: 1.2780 - val_accuracy: 0.4977
Epoch 4/10
439/439 [==============================] - 28s 63ms/step - loss: 1.0426 - accuracy: 0.5967 - val_loss: 1.1051 - val_accuracy: 0.5823
Epoch 5/10
439/439 [==============================] - 32s 73ms/step - loss: 0.9982 - accuracy: 0.6213 - val_loss: 0.9316 - val_accuracy: 0.6293
Epoch 6/10
439/439 [==============================] - 31s 70ms/step - loss: 0.9479 - accuracy: 0.6371 - val_loss: 0.8866 - val_accuracy: 0.6700
Epoch 7/10
439/439 [==============================] - 32s 74ms/step - loss: 0.9158 - accuracy: 0.6566 - val_loss: 1.0009 - val_accuracy: 0.6157
Epo

In [11]:
accuracy=cnn.evaluate(test_set, steps=len(test_set),verbose=2)
print('Accuracy of the model on the test set: ',accuracy[1])

94/94 - 2s - loss: 0.8702 - accuracy: 0.6807 - 2s/epoch - 23ms/step
Accuracy of the model on the test set:  0.6806666851043701
